In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "drive/My Drive/train.zip"

In [ ]:
!unzip "drive/My Drive/val.zip"

In [2]:
import keras,os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np

# base_dir = '/root/Project'

# train_dir = os.path.join(base_dir, 'train')
# validation_dir = os.path.join(base_dir, 'val')
# test_dir = os.path.join(base_dir, 'test')

train_dir = 'train'
validation_dir = 'val'
test_dir = 'test'

# Directory with our training ad pictures
train_ads_dir = os.path.join(train_dir, 'pos')

# Directory with our training dog pictures
train_noads_dir = os.path.join(train_dir, 'neg')

# Directory with our validation ad pictures
validation_ads_dir = os.path.join(validation_dir, 'pos')

# Directory with our validation dog pictures
validation_noads_dir = os.path.join(validation_dir, 'neg')


In [ ]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [ ]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20, class_mode = 'binary', target_size = (300, 300))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory( validation_dir,  batch_size = 20, class_mode = 'binary', target_size = (300, 300))

Found 47057 images belonging to 2 classes.
Found 15685 images belonging to 2 classes.


In [ ]:
from keras.optimizers import Adam
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten , Dropout
from keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (300, 300, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False
    
# Flatten the output layer to 1 dimension
x = Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = Dense(1, activation='sigmoid')(x)

model = keras.models.Model(base_model.input, x)

model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
vgghist = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)

Epoch 1/10
100/100 [==============================] - ETA: 0s - loss: 0.4534 - acc: 0.8260

/usr/local/lib/python3.8/dist-packages/PIL/Image.py:2797: DecompressionBombWarning: Image size (141750000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


100/100 [==============================] - 435s 4s/step - loss: 0.4534 - acc: 0.8260 - val_loss: 0.3826 - val_acc: 0.8569
Epoch 2/10
100/100 [==============================] - 422s 4s/step - loss: 0.3954 - acc: 0.8405 - val_loss: 0.3204 - val_acc: 0.8808
Epoch 3/10
100/100 [==============================] - 419s 4s/step - loss: 0.3480 - acc: 0.8640 - val_loss: 0.3417 - val_acc: 0.8578
Epoch 4/10
100/100 [==============================] - 401s 4s/step - loss: 0.3536 - acc: 0.8630 - val_loss: 0.3127 - val_acc: 0.8755
Epoch 5/10
100/100 [==============================] - 394s 4s/step - loss: 0.3359 - acc: 0.8575 - val_loss: 0.3005 - val_acc: 0.8890
Epoch 6/10
100/100 [==============================] - 391s 4s/step - loss: 0.3109 - acc: 0.8760 - val_loss: 0.3005 - val_acc: 0.8843
Epoch 7/10
100/100 [==============================] - 391s 4s/step - loss: 0.3316 - acc: 0.8720 - val_loss: 0.3296 - val_acc: 0.8618
Epoch 8/10
100/100 [==============================] - 393s 4s/step - loss: 0.347

In [3]:
!unzip "drive/My Drive/test.zip"

Streaming output truncated to the last 5000 lines.
  inflating: test/pos/66326.jpg      
  inflating: test/pos/171447.png     
  inflating: test/pos/171153.png     
  inflating: test/pos/172639.png     
  inflating: test/pos/87071.jpg      
  inflating: test/pos/147858.jpg     
  inflating: test/pos/16323.jpg      
  inflating: test/pos/122819.jpg     
  inflating: test/pos/27074.jpg      
  inflating: test/pos/80673.jpg      
  inflating: test/pos/32418.jpg      
  inflating: test/pos/131125.jpg     
  inflating: test/pos/135658.jpg     
  inflating: test/pos/14252.jpg      
  inflating: test/pos/78996.jpg      
  inflating: test/pos/115560.jpg     
  inflating: test/pos/14667.jpg      
  inflating: test/pos/28400.jpg      
  inflating: test/pos/92392.jpg      
  inflating: test/pos/176557.png     
  inflating: test/pos/97073.jpg      
  inflating: test/pos/6768.jpg       
  inflating: test/pos/5263.jpg       
  inflating: test/pos/108570.jpg     
  inflating: test/pos/176280.png     

In [11]:
from keras.models import load_model

# Save model
model.save("Ad_NoAd_VGG2.h5")

In [4]:
from keras.models import load_model
from PIL import Image

Image.MAX_IMAGE_PIXELS = None

# Flow test images in batches of 20 using test_datagen generator
test_gen = ImageDataGenerator( rescale = 1.0/255. )
test_generator = test_gen.flow_from_directory( test_dir,  batch_size = 20, class_mode = 'binary', target_size = (300, 300))
saved_model = load_model("Ad_NoAd_VGG.h5")

# Evaluate on test data
scores = saved_model.evaluate(test_generator)
print("%s%s: %.2f%%" % ("evaluate ",saved_model.metrics_names[1], scores[1]*100))

Found 15687 images belonging to 2 classes.
785/785 [==============================] - 340s 420ms/step - loss: 0.3046 - acc: 0.8757
evaluate acc: 87.57%


In [12]:
from sklearn.metrics import confusion_matrix

# y_pred = saved_model.predict(test_generator)
y_pred = saved_model.predict(test_generator)

785/785 [==============================] - 299s 381ms/step


In [25]:
y_pred = np.rint(y_pred)
print(confusion_matrix(test_generator.classes, y_pred))

[[  158  2562]
 [  788 12179]]


In [37]:
print(test_generator.classes)
print(test_generator.class_indices.keys())

[0 0 0 ... 1 1 1]
dict_keys(['neg', 'pos'])


In [35]:
from sklearn.metrics import classification_report

print(classification_report(test_generator.classes, y_pred))

              precision    recall  f1-score   support

           0       0.17      0.06      0.09      2720
           1       0.83      0.94      0.88     12967

    accuracy                           0.79     15687
   macro avg       0.50      0.50      0.48     15687
weighted avg       0.71      0.79      0.74     15687

